## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-16-06-43-48 +0000,nypost,Manhattan DA’s office to petition Supreme Cour...,https://nypost.com/2025/09/16/us-news/manhatta...
1,2025-09-16-06-38-22 +0000,bbc,State pension likely to rise by 4.7% in April,https://www.bbc.com/news/articles/c62lnzdndkeo...
2,2025-09-16-06-38-00 +0000,wsj,"U.K.’s Jobless Rate Unchanged, Keeping BOE on ...",https://www.wsj.com/economy/central-banking/u-...
3,2025-09-16-06-35-00 +0000,wsj,Israel Launches New Ground Offensive in Bid to...,https://www.wsj.com/world/middle-east/israel-l...
4,2025-09-16-06-33-27 +0000,bbc,Malawi votes for next president as cost-of-liv...,https://www.bbc.com/news/articles/ckgy3g7zpl3o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
97,trump,56
46,kirk,31
45,charlie,31
29,new,20
158,china,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
66,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...,129
67,2025-09-16-00-39-31 +0000,nypost,Senate confirms Trump Fed pick Stephen Miran a...,https://nypost.com/2025/09/15/us-news/senate-c...,114
55,2025-09-16-01-14-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...,113
16,2025-09-16-04-57-35 +0000,bbc,Trump says he will sue New York Times for $15bn,https://www.bbc.com/news/articles/cevz419pk8vo...,110
20,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...,110


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
66,129,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...
20,110,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...
55,57,2025-09-16-01-14-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...
3,53,2025-09-16-06-35-00 +0000,wsj,Israel Launches New Ground Offensive in Bid to...,https://www.wsj.com/world/middle-east/israel-l...
107,47,2025-09-15-22-18-14 +0000,nypost,"US, China near TikTok deal to finally resolve ...",https://nypost.com/2025/09/15/business/us-chin...
143,41,2025-09-15-20-15-10 +0000,nypost,Trump urges Republicans to back stopgap bill a...,https://nypost.com/2025/09/15/us-news/trump-ca...
224,41,2025-09-15-15-15-00 +0000,wsj,Canadian factories notched the biggest rise in...,https://www.wsj.com/articles/canada-factory-an...
232,39,2025-09-15-14-39-54 +0000,nypost,Tesla share surge as Elon Musk buys $1B in sto...,https://nypost.com/2025/09/15/business/tesla-s...
245,34,2025-09-15-14-14-04 +0000,nypost,Charlie Kirk shooting is ‘the death of free sp...,https://nypost.com/2025/09/15/media/charlie-ki...
135,33,2025-09-15-20-29-55 +0000,cbc,"Carney, Poilievre come face-to-face for the fi...",https://www.cbc.ca/news/politics/carney-poilie...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
